# Backup Validation and DBCC CHECKDB using Rubrik Live Mount

## Backup Validatation of a database using Live Mount

Live mount allows for near instant recovery of a database. If a database restore/export normally takes hours, then live mounting a database will take a few minutes. Live Mount does a full recovery of a database to either the same SQL Server Instance with a different database name or another SQL Server Instance with the same or different database name. The recovery of the database is much faster, because Rubrik does not need to copy the contents of the backup from the Rubrik Cluster back to the SQL Server. All of the recovery work is done on the Rubrik cluster itself. Then the database files are presented to the SQL Server Instance via a secure SMB3 share that is only accessible by the machine the share is mounted to.

Live Mounting a database is great for a lot of different use cases:
- DBA Backup validation testing
- Object level recovery
- Developer testing
- DevOps Automation
- Reporting databases
- Database migration application smoke test validation.

A key parameter is RecoveryDateTime. All dates in Rubrik are stored in UTC format. This parameter is expecting a fully qualified date and time in UTC format. example value is 2018-08-01T02:00:00.000Z. In the example below, we are pulling the latest recovery point that Rubrik knows about.

**This article serves as a way to demonstrate how to use Live Mount for Backup Validation.**

### Set up environment for all next steps. 

##### Install the SQLServer package

In [ ]:
# Set Variables
$Server = $Rubrik.Server.cdm02
$Token = $Rubrik.token.cdm02
$SourceSQLServerInstance = "rp-sql19s-001.perf.rubrik.com"
$SourceDatabaseName = "AdventureWorks2019"
$TargetSQLServerInstance = "rp-sql19s-001.perf.rubrik.com"
$MountedDatabaseName = "AdventureWorks2019_LiveMount"

#### Connect to the Rubrik Cluster

In [ ]:
#Connect-Rubrik with an API Token
Connect-Rubrik -Server $Server -Token $Token


Name                           Value
----                           -----
api                            1
version                        6.0.2-p2-13398
server                         10.8.49.101
header                         {User-Agent, Authorization}
id                             
authType                       Token
time                           01/13/2022 18:34:50
userId                         5e8de809-821c-4569-b53b-8bc3aa5b2f2a



#### Get details about the database from the Rubrik Cluster

In [ ]:
# Get database information from Rubrik
$RubrikDatabase = Get-RubrikDatabase -Name $SourceDatabaseName -ServerInstance $SourceSQLServerInstance
$RubrikDatabase | Format-List *


id                                   : MssqlDatabase:::42527305-5e85-4777-8689-955237d3d951
name                                 : AdventureWorks2019
configuredSlaDomainId                : INHERIT
configuredSlaDomainName              : Inherit
configuredSlaDomainType              : ProtectionSla
primaryClusterId                     : 39b92c18-d897-4b55-a7f9-17ff178616d0
isConfiguredSlaDomainRetentionLocked : False
slaLastUpdateTime                    : 08/06/2021 14:05:30
effectiveSlaDomainId                 : 4be3bc1f-6936-432e-86ef-220554df0c35
effectiveSlaDomainName               : SQL Server
isEffectiveSlaDomainRetentionLocked  : False
effectiveSlaSourceObjectId           : MssqlInstance:::8c66675e-8c84-45b6-a136-73dce79d2372
effectiveSlaSourceObjectName         : MSSQLSERVER
slaAssignment                        : Derived
retentionSlaDomainId                 : 4be3bc1f-6936-432e-86ef-220554df0c35
rootProperties                       : @{rootType=Host; rootId=Host:::93c5aed0-2140-4

#### Mount the database to a SQL Server

In [ ]:
#Mount a database to a SQL Server
$TargetInstance = Get-RubrikSQLInstance -ServerInstance $TargetSQLServerInstance
$RubrikRequest = New-RubrikDatabaseMount -id $RubrikDatabase.id `
	-TargetInstanceId $TargetInstance.id `
	-MountedDatabaseName $MountedDatabaseName `
	-recoveryDateTime (Get-date (Get-RubrikDatabase -id $RubrikDatabase.id).latestRecoveryPoint) `
    -Confirm:$false
Get-RubrikRequest -id $RubrikRequest.id -Type mssql -WaitForCompletion    


id        : MSSQL_DB_MOUNT_23a5e731-04ef-48ef-9c77-4e8a89db4e9b_4f453e22-eeb9-489a-acaf-d665403fc08
            d:::0
status    : SUCCEEDED
startTime : 01/13/2022 23:34:59
endTime   : 01/13/2022 23:35:43
nodeId    : cluster:::RVMHM204S006832
links     : {@{href=https://10.8.49.101/api/v1/mssql/db/mount/5642b730-80d1-4e42-8ae6-8fc326eb2dc5;
             rel=result}, @{href=https://10.8.49.101/api/v1/mssql/request/MSSQL_DB_MOUNT_23a5e731-0
            4ef-48ef-9c77-4e8a89db4e9b_4f453e22-eeb9-489a-acaf-d665403fc08d:::0; rel=self}}




A Live mount of a database is the equivalent to doing a T-SQL Restore with your native backups. SQL Server has recovered the snapshot via the SQL Server VSS Writer, and if applicable, rolled the database forward to a point in time chosen by the user. This means we have applied all transactions from the time the snapshot has happened until the point in time chosen. Once a database has been Live Mounted to a SQL Server, the database is ready for any read/write query you would like to run. 

In [ ]:
$Query = "SELECT name, state_desc FROM sys.databases"
Invoke-Sqlcmd -ServerInstance $TargetSQLServerInstance -Query $Query | Format-Table


name                         state_desc
----                         ----------
master                       ONLINE
tempdb                       ONLINE
model                        ONLINE
msdb                         ONLINE
AdventureWorks2019           ONLINE
AdventureWorksDW2019         ONLINE
AdventureWorksLT2019         ONLINE
AdventureWorks2019_LiveMount ONLINE



In [ ]:
#Unmount a database from SQL Server
$RubrikDatabaseMount = Get-RubrikDatabaseMount -MountedDatabaseName $MountedDatabaseName -TargetInstanceId $TargetInstance.id
$RubrikRequest = Remove-RubrikDatabaseMount -id $RubrikDatabaseMount.id -Confirm:$false